# Challenge 6: Personalized Insurance Recommendations

## Introduction
Welcome to Challenge 6! In this challenge, we will guide you through building a personalized insurance recommendation system. Even if many of these concepts are new, don’t worry – each step is explained in detail.

In this challenge, you'll learn how to:
* **Retrieve Data:** Query and process loan application data from a Cosmos DB. Cosmos DB is a globally distributed NoSQL database that stores data in JSON format.
* **Generate Recommendations:** Use basic business logic to generate insurance recommendations based on loan details like the loan amount, type, and the borrower’s age.
* **AI Integration:** Integrate our functions with Semantic Kernel—a tool that helps organize tasks and calls intelligent services (powered by Azure OpenAI) to produce automated recommendations.

Each step includes detailed inline commentary to help you understand what is happening, so take your time reading through the comments!

## Step 1: Environment Setup

In this first step, we import the required Python libraries and set up our connections to the Azure services. Here’s what happens:
- **semantic_kernel:** A library that helps combine functions into a single intelligent agent.
- **Azure OpenAI Chat Service:** Connects us to an AI model that will help generate recommendations.
- **Azure Cosmos DB client:** Used to securely connect to our database which contains your loan documents in JSON format.
- **dotenv:** Loads environment variables from a `.env` file. Ensure that your `.env` file includes keys such as `COSMOS_ENDPOINT`, `COSMOS_KEY`, `AZURE_OPENAI_DEPLOYMENT_NAME`, `AZURE_OPENAI_KEY`, and `AZURE_OPENAI_ENDPOINT`.

With these environment settings, your code will securely authenticate with the relevant services and remain portable.

In [1]:
from azure.cosmos import CosmosClient, PartitionKey
import os
import json
from dotenv import load_dotenv
from typing import Annotated

# Load environment variables
load_dotenv()

# Azure service configuration
cosmos_endpoint = os.getenv('COSMOS_ENDPOINT')
cosmos_key = os.getenv('COSMOS_KEY')
database_name = "ContosoDB"
openai_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
openai_api_key = os.getenv("AZURE_OPENAI_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")

# Initialize Cosmos client
client = CosmosClient(cosmos_endpoint, cosmos_key)
database = client.get_database_client(database_name)

## Step 2: Data Retrieval Functions

We now define functions to query our Cosmos DB for specific types of data. This step is critical because:
- **`get_loan_applicant_info`:** This function retrieves a loan application by matching the applicant's name from the 'LoanForms' container.
- **`get_paystub_info`:** This function queries the 'PayStubs' container in Cosmos DB to get paystub information using the employee's name.

Both functions use try/except blocks so that any errors (such as connection issues) are caught and printed. Detailed inline comments in the code will explain each part of the query process.

In [27]:
#First we install the required libraries

!pip install semantic-kernel azure-identity azure-cosmos python-dotenv

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [28]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import ChatHistory
from semantic_kernel.connectors.ai import FunctionChoiceBehavior

In [3]:
@kernel_function(description="Retrieves loan application information for a given applicant.", name="get_loan_applicant_info")
def get_loan_applicant_info(applicant_name: Annotated[str, "The name of the loan applicant to retrieve information for."]) -> Annotated[str, "The JSON string containing loan application data."]:
    """Retrieves loan application information for a given applicant.
    
    Args:
        applicant_name (str): Name of the loan applicant
        
    Returns:
        str: JSON string containing loan application data
    """
    try:
        container = database.get_container_client("LoanForms")
        query = f"SELECT * FROM c WHERE c.content[0][\"Full Name\"] = '{applicant_name}'"
        items = list(container.query_items(query, enable_cross_partition_query=True))
        return json.dumps(items[0]) if items else "{}"
    except Exception as e:
        print(f"Error retrieving loan application: {str(e)}")
        return "{}"

In [4]:
@kernel_function(description="Retrieves paystub information for a given employee.", name="get_paystub_info")
def get_paystub_info(employee_name: Annotated[str, "The name of the employee to retrieve paystub information for."]) -> Annotated[str, "The JSON string containing paystub data."]:
    """Retrieves paystub information for a given employee.
    
    Args:
        employee_name (str): Name of the employee
        
    Returns:
        str: JSON string containing paystub data
    """
    try:
        container = database.get_container_client("PayStubs")
        query = f"SELECT * FROM c WHERE c.content['pay stub details']['Employee Name'] = '{employee_name}'"
        items = list(container.query_items(query, enable_cross_partition_query=True))
        return json.dumps(items) if items else f"No paystub information found for {employee_name}."
    except Exception as e:
        print(f"Error retrieving paystub: {str(e)}")
        return ""

In [5]:
get_loan_applicant_info("Jane Elizabeth Smith")

'{"id": "100001", "content": [{"id": "100001", "Full Name": "Jane Elizabeth Smith", "Date of Birth": "08/22/1990", "Social Security Number": "987-65-4321", "Contact Number": "(555) 234-5678", "Email Address": "jane.smith90@example.com", "Physical Address": "456 Oak Avenue Unit 10 Madison WI 53703", "Loan Amount Requested": "40000", "Purpose of Loan": "Vehicle Purchase", "Loan Term Desired": "5 years Applicant\'s Signature: Jz", "Employer Name": "Horizon Retailers", "Position": "Store Manager ", "Employment Duration": "3 years ", "Monthly Income": "$4583.33 ", "Employer Contact Number": "(555) 789- 2345"}], "_rid": "R-YcAONky8EBAAAAAAAAAA==", "_self": "dbs/R-YcAA==/colls/R-YcAONky8E=/docs/R-YcAONky8EBAAAAAAAAAA==/", "_etag": "\\"e100150c-0000-4700-0000-67b0bfbf0000\\"", "_attachments": "attachments/", "_ts": 1739636671}'

In [6]:
# retrieve loan agreement
@kernel_function(description="Retrieves loan agreement information for a given applicant.", name="get_loan_agreement_info")
def get_loan_agreement_info(applicant_name: Annotated[str, "The name of the loan applicant to retrieve agreement information for."]) -> Annotated[str, "The JSON string containing loan agreement data."]:
    """Retrieves loan agreement information for a given applicant.
    
    Args:
        applicant_name (str): Name of the loan applicant
    
    Returns:
        str: JSON string containing loan agreement data
    """
    try:
        container = database.get_container_client("LoanAgreements")
        query = f"SELECT * FROM c WHERE c.content.borrower_informaiton like '%{applicant_name}%'"
        items = list(container.query_items(query, enable_cross_partition_query=True))
        return json.dumps(items[0]) if items else "{}"
    except Exception as e:
        print(f"Error retrieving loan agreement: {str(e)}")
        return "{}"
    

## Step 3: Insurance Recommendation Function

This cell introduces the `recommend_insurance` function. Here’s what it does:
- It accepts the loan amount, type, and the borrower’s age as inputs.
- It converts the loan amount (given as a string like "$20,000") into a numeric value so that comparisons can be made.
- Using conditional logic, it returns a recommended set of insurance products. For example, for larger personal loans or mortgages, different recommendations are made.

Remember: This is a simplified example. In production, you might have more complex rules. The inline comments explain every branch of the logic to help you follow along.

In [7]:
@kernel_function(description="Recommends insurance products based on loan details and borrower profile.", name="recommend_insurance")
def recommend_insurance(loan_amount: Annotated[str, "The amount of the loan as a string with a dollar sign and commas."], loan_type: Annotated[str, "The type of loan (e.g., 'Personal Loan', 'Mortgage', 'Auto Loan', 'Business Loan', 'Student Loan')."], borrower_age: Annotated[int, "The age of the borrower."]) -> Annotated[str, "The recommended insurance products as a string."]:
    """Recommends insurance products based on loan details and borrower profile.

    Args:
        loan_amount (str): Amount of the loan
        loan_type (str): Type of loan (Personal Loan, Mortgage, Auto Loan, Business Loan, or Student Loan)
        borrower_age (int): Age of the borrower

    Returns:
        str: Insurance recommendation message
    """
    try:
        loan_amount_value = float(
            loan_amount.replace('$', '').replace(',', ''))

        if loan_type == "Personal Loan":
            if loan_amount_value > 10000:
                if borrower_age > 50:
                    return "Recommended Insurance: Contoso LifeGuard Plus, Contoso Personal Shield Elite"
                return "Recommended Insurance: Contoso Personal Shield Standard"
            return "Recommended Insurance: Contoso Personal Shield Basic"

        elif loan_type == "Mortgage":
            if loan_amount_value > 200000:
                return "Recommended Insurance: Contoso HomeGuard Premium, Contoso MortgageShield Plus"
            return "Recommended Insurance: Contoso HomeGuard Essential, Contoso MortgageShield Basic (optional)"

        elif loan_type == "Auto Loan":
            if loan_amount_value > 30000:
                return "Recommended Insurance: Contoso AutoGuard Elite, Contoso ValueGap Plus, Contoso ExtendedCare Premium"
            elif loan_amount_value > 15000:
                return "Recommended Insurance: Contoso AutoGuard Plus, Contoso ValueGap Standard"
            return "Recommended Insurance: Contoso AutoGuard Basic"

        elif loan_type == "Business Loan":
            if loan_amount_value > 100000:
                return "Recommended Insurance: Contoso BusinessShield Premium, Contoso KeyTalent Protect, Contoso BusinessContinuity Plus"
            return "Recommended Insurance: Contoso BusinessShield Essential"

        elif loan_type == "Student Loan":
            if borrower_age < 25:
                return "Recommended Insurance: Contoso StudentGuard Plus, Contoso HealthEssentials"
            return "Recommended Insurance: Contoso StudentGuard Basic"

        return "Recommended Insurance: Schedule a consultation with a Contoso Insurance Advisor for personalized recommendations."
    except ValueError as e:
        print(f"Error processing loan amount: {str(e)}")
        return "Error: Unable to process loan amount"
    except Exception as e:
        print(f"Error generating recommendation: {str(e)}")
        return "Error: Unable to generate recommendation"

## Deep Dive: What is Semantic Kernel?

Semantic Kernel is a powerful, modular SDK developed by Microsoft that enables you to easily integrate advanced AI models into your applications. It acts as a middleware layer that enables your native code to interact with external AI services. Here are the key components and ideas behind it:

- **Kernel:** The heart of the Semantic Kernel. This is where you register all your connectors (which link your application to AI services) and plugins (which are modular units of native code that execute specific tasks). The kernel orchestrates all the interactions required by your AI workflow.

- **Agents:** Instead of relying on rigid sequential workflows, AI agents enable dynamic decision-making. Agents can interactively synthesize responses by invoking registered functions on-the-fly, enabling flexible and context-aware responses.

- **Connectors:** These are the bridges between your application and remote AI services—for example, the Azure OpenAI Chat service. They handle the communication, data formatting, and authentication required to make API calls.

- **Plugins (or Skills):** Modular functions that perform specific tasks. By grouping related native functions into plugins, the AI component can invoke these functions seamlessly during execution.

The strength of Semantic Kernel lies in its ability to combine your existing code with state-of-the-art AI models. It’s designed to be flexible and future‑proof: whether you’re working with models from Microsoft, OpenAI, or even Hugging Face, you can swap them out without rewriting your entire codebase.

## Step 4: Semantic Kernel Integration

Semantic Kernel is a framework that helps us orchestrate multiple functions into an intelligent workflow. In this step:
- We initialize the Semantic Kernel instance.
- We add the Azure OpenAI Chat service to enable intelligent text processing.
- Our custom functions (`get_loan_applicant_info`, `get_paystub_info`, and `recommend_insurance`) are registered as a plugin so that the Semantic Kernel can use them to plan and execute tasks.

If you’re new to concepts like async programming (you’ll see `await` later) or AI orchestration, read through the inline comments—the code walks you through how these components are connected.

In [8]:
# Initialize Semantic Kernel
kernel = sk.Kernel()

# Add Azure OpenAI service
deployment = AzureChatCompletion(
    service_id="default",
    deployment_name="gpt-4o",
    endpoint=openai_endpoint,
    api_key=openai_api_key
)
kernel.add_service(deployment)

# Register functions as plugin
plugin = kernel.add_functions(
    plugin_name="LoanAndInsurancePlugin",
    functions=[
        get_loan_applicant_info,
        get_paystub_info,
        get_loan_agreement_info,
        recommend_insurance
    ]
)

## Step 5: Create and Execute Recommendation Agent

Now we define an intelligent agent powered by Semantic Kernel leveraging the Azure OpenAI Chat service to handle the recommendation process. The agent is configured with detailed instructions and a conversation history that includes the customer query:
"Please retrieve loan application and paystub information for 'Jane Elizabeth Smith' and recommend relevant insurance products."

Instead of using a sequential planner to decompose the task into ordered steps, the agent dynamically synthesizes responses by invoking the appropriate registered functions based on the provided context. When invoked asynchronously, the agent processes the conversation and returns a personalized insurance recommendation crafted with a friendly and professional tone.

In [9]:
# Get current date
current_date = datetime.now().strftime("%Y-%m-%d")

# Define agent with name and instructions
AGENT_NAME = "insurance_advisor" # Changed to use underscores instead of spaces to match pattern
AGENT_INSTRUCTIONS = f"""You are an insurance advisor that helps recommend insurance products based on loan applications.
Current date: {current_date}

RULES:
1. You should always recommend insurance products with information retrieved from 'recommend_insurance' function.
2. You will craft the message to be sent to the user, and your response should contain the crafted message inside <message> tags.
3. Outside of the <message> tags, you should include other information that is not the crafted message, such as the reasoning behind the recommendation.
4. The message tone should be friendly and professional, and the language should be engaging and easy to understand and aligned with Contoso Insurance brand voice.
"""

settings = kernel.get_prompt_execution_settings_from_service_id(service_id="default")
# Configure the function choice behavior to auto invoke kernel functions
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

agent = ChatCompletionAgent(service_id="default", kernel=kernel, name=AGENT_NAME, arguments=KernelArguments(settings=settings))

# Define the chat history
chat_history = ChatHistory()
chat_history.add_system_message(AGENT_INSTRUCTIONS)

# Add user request
user_input = "Please retrieve loan application and paystub information for 'Jane Elizabeth Smith' and recommend relevant insurance products."
chat_history.add_user_message(user_input)

print("Invoking insurance advisor agent...")
async for content in agent.invoke(chat_history):
    chat_history.add_message(content)
    print(f"# Agent - {content.name or '*'}: '{content.content}'")

Invoking insurance advisor agent...
# Agent - insurance_advisor: 'Based on the information provided, Jane Elizabeth Smith has applied for a vehicle loan of $40,000 at age 34 and has a stable career as a store manager earning a consistent income. Contoso Insurance has tailored an insurance recommendation to help protect her financial investment.

<message>
Hi Jane, 

Congratulations on your loan application for your new vehicle! As you prepare for this exciting purchase, Contoso Insurance recommends scheduling a consultation with one of our experienced advisors. We can assist you in selecting insurance coverage tailored to your needs—whether it’s comprehensive auto insurance, gap coverage, or loan protection. We’re here to ensure your financial well-being remains secure. 

Let us know a convenient time for you to chat about the best plan to safeguard your investment. We’re looking forward to helping you drive into the future confidently!

Best regards,  
The Contoso Insurance Team
</mes

In [10]:
# parse the response to get the message that is inside <message> tags
import re
from IPython.display import display, Markdown


# Extract the message from the response
message_pattern = r'<message>(.*?)</message>'
message_match = re.search(message_pattern, chat_history.messages[-1].content, re.DOTALL)

display(Markdown(message_match.group(1)))


Hi Jane, 

Congratulations on your loan application for your new vehicle! As you prepare for this exciting purchase, Contoso Insurance recommends scheduling a consultation with one of our experienced advisors. We can assist you in selecting insurance coverage tailored to your needs—whether it’s comprehensive auto insurance, gap coverage, or loan protection. We’re here to ensure your financial well-being remains secure. 

Let us know a convenient time for you to chat about the best plan to safeguard your investment. We’re looking forward to helping you drive into the future confidently!

Best regards,  
The Contoso Insurance Team


Now that you have seen the AI Agent in action, try to modify the agent's instructions and get it to create different messages, like 

## Conclusion

In this challenge, you learned how to:
* **Retrieve and Process Data:** Extract loan application and paystub information from Cosmos DB.
* **Generate Recommendations:** Use simple business logic to make personalized insurance recommendations based on key factors like loan amount, loan type, and borrower age.
* **Integrate AI:** Combine your custom functions with Semantic Kernel to orchestrate this workflow and enable intelligent task planning.

This challenge might introduce several new technologies and programming concepts, such as working with asynchronous code, integrating external services, and orchestrating complex workflows. Each step is carefully annotated with explanations and inline comments, so refer to these comments if you get stuck or want to understand the logic behind each operation.

By the end of this challenge, you should have a strong understanding of how structured data and AI services can be combined to build intelligent applications that automate decision-making processes.